# Forex Market Trend Analysis Tools

This notebook implements three powerful technical analysis tools for analyzing forex market trends:

1. Moving Average Crossover System
2. Relative Strength Index (RSI) Analysis
3. Bollinger Bands with Volume Analysis

Each tool provides different insights into market trends and can be used in combination for more effective trading decisions.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from datetime import datetime
import os
import git

# Set up logging
log_dir = 'logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

logging.basicConfig(
    filename=os.path.join(log_dir, f'forex_analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logging.info('Starting Forex Market Analysis')

# Tool 1: Moving Average Crossover System

The Moving Average Crossover system is one of the most popular trend-following strategies. It uses two moving averages:
- Short-term MA (faster)
- Long-term MA (slower)

When the faster MA crosses above the slower MA, it generates a buy signal (bullish trend).
When the faster MA crosses below the slower MA, it generates a sell signal (bearish trend).

This system helps identify trend direction and potential entry/exit points.

In [ ]:
def calculate_ma_crossover(data, short_window=20, long_window=50):
    """
    Calculate Moving Average Crossover signals
    
    Parameters:
    data (pd.DataFrame): DataFrame with 'Close' price column
    short_window (int): Short-term moving average period
    long_window (int): Long-term moving average period
    
    Returns:
    pd.DataFrame: DataFrame with MA values and signals
    """
    logging.info(f'Calculating MA Crossover with windows: {short_window}, {long_window}')
    
    # Calculate moving averages
    data['SMA_Short'] = data['Close'].rolling(window=short_window).mean()
    data['SMA_Long'] = data['Close'].rolling(window=long_window).mean()
    
    # Generate signals
    data['Signal'] = 0
    data.loc[data['SMA_Short'] > data['SMA_Long'], 'Signal'] = 1  # Buy signal
    data.loc[data['SMA_Short'] < data['SMA_Long'], 'Signal'] = -1  # Sell signal
    
    # Calculate crossover points
    data['Signal_Change'] = data['Signal'].diff()
    
    logging.info('MA Crossover calculation completed')
    return data

# Example usage with sample data
# Load your forex data here
sample_data = pd.DataFrame({
    'Close': np.random.randn(1000).cumsum()
}, index=pd.date_range(start='2024-01-01', periods=1000))

# Calculate MA Crossover
result = calculate_ma_crossover(sample_data.copy())

# Plotting
plt.figure(figsize=(15, 7))
plt.plot(result.index, result['Close'], label='Price')
plt.plot(result.index, result['SMA_Short'], label=f'Short MA (20)')
plt.plot(result.index, result['SMA_Long'], label=f'Long MA (50)')

# Plot buy/sell signals
buy_signals = result[result['Signal_Change'] == 2].index
sell_signals = result[result['Signal_Change'] == -2].index

plt.scatter(buy_signals, result.loc[buy_signals, 'Close'], 
           marker='^', color='green', label='Buy Signal')
plt.scatter(sell_signals, result.loc[sell_signals, 'Close'],
           marker='v', color='red', label='Sell Signal')

plt.title('Moving Average Crossover Strategy')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

logging.info(f'Found {len(buy_signals)} buy signals and {len(sell_signals)} sell signals')

# Tool 2: Relative Strength Index (RSI) Analysis

The RSI is a momentum oscillator that measures the speed and magnitude of recent price changes to evaluate overbought or oversold conditions.

Key characteristics:
- RSI ranges from 0 to 100
- Traditional overbought level: 70
- Traditional oversold level: 30
- Divergence between RSI and price can signal potential trend reversals

In [ ]:
def calculate_rsi(data, period=14):
    """
    Calculate Relative Strength Index
    
    Parameters:
    data (pd.DataFrame): DataFrame with 'Close' price column
    period (int): RSI calculation period
    
    Returns:
    pd.DataFrame: DataFrame with RSI values and signals
    """
    logging.info(f'Calculating RSI with period: {period}')
    
    # Calculate price changes
    delta = data['Close'].diff()
    
    # Separate gains and losses
    gain = (delta.where(delta > 0, 0))
    loss = (-delta.where(delta < 0, 0))
    
    # Calculate average gain and loss
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    # Calculate RS and RSI
    rs = avg_gain / avg_loss
    data['RSI'] = 100 - (100 / (1 + rs))
    
    # Generate signals
    data['RSI_Signal'] = 0
    data.loc[data['RSI'] < 30, 'RSI_Signal'] = 1  # Oversold - Buy signal
    data.loc[data['RSI'] > 70, 'RSI_Signal'] = -1  # Overbought - Sell signal
    
    logging.info('RSI calculation completed')
    return data

# Calculate RSI for our sample data
result = calculate_rsi(sample_data.copy())

# Plotting
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), gridspec_kw={'height_ratios': [2, 1]})

# Plot price
ax1.plot(result.index, result['Close'], label='Price')
ax1.set_title('Price and RSI Analysis')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price')
ax1.grid(True)
ax1.legend()

# Plot RSI
ax2.plot(result.index, result['RSI'], label='RSI', color='purple')
ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5)
ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5)
ax2.fill_between(result.index, 70, 100, color='r', alpha=0.1)
ax2.fill_between(result.index, 0, 30, color='g', alpha=0.1)
ax2.set_ylabel('RSI')
ax2.set_xlabel('Date')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

# Log analysis results
oversold_count = len(result[result['RSI'] < 30])
overbought_count = len(result[result['RSI'] > 70])
logging.info(f'Found {oversold_count} oversold and {overbought_count} overbought conditions')

# Tool 3: Bollinger Bands Analysis

Bollinger Bands consist of three lines:
1. Middle Band: 20-day simple moving average (SMA)
2. Upper Band: SMA + (2 × standard deviation)
3. Lower Band: SMA - (2 × standard deviation)

These bands help identify:
- Market volatility (band width)
- Potential price breakouts
- Mean reversion opportunities
- Trend strength

In [ ]:
def calculate_bollinger_bands(data, window=20, num_std=2):
    """
    Calculate Bollinger Bands
    
    Parameters:
    data (pd.DataFrame): DataFrame with 'Close' price column
    window (int): Moving average window
    num_std (int): Number of standard deviations for bands
    
    Returns:
    pd.DataFrame: DataFrame with Bollinger Bands values and signals
    """
    logging.info(f'Calculating Bollinger Bands with window: {window}, std: {num_std}')
    
    # Calculate middle band (SMA)
    data['BB_Middle'] = data['Close'].rolling(window=window).mean()
    
    # Calculate standard deviation
    rolling_std = data['Close'].rolling(window=window).std()
    
    # Calculate upper and lower bands
    data['BB_Upper'] = data['BB_Middle'] + (rolling_std * num_std)
    data['BB_Lower'] = data['BB_Middle'] - (rolling_std * num_std)
    
    # Calculate bandwidth (volatility indicator)
    data['BB_Width'] = (data['BB_Upper'] - data['BB_Lower']) / data['BB_Middle']
    
    # Generate signals
    data['BB_Signal'] = 0
    data.loc[data['Close'] > data['BB_Upper'], 'BB_Signal'] = -1  # Overbought
    data.loc[data['Close'] < data['BB_Lower'], 'BB_Signal'] = 1   # Oversold
    
    logging.info('Bollinger Bands calculation completed')
    return data

# Calculate Bollinger Bands for our sample data
result = calculate_bollinger_bands(sample_data.copy())

# Plotting
plt.figure(figsize=(15, 7))

# Plot price and bands
plt.plot(result.index, result['Close'], label='Price', color='blue')
plt.plot(result.index, result['BB_Middle'], label='Middle Band', color='gray', linestyle='--')
plt.plot(result.index, result['BB_Upper'], label='Upper Band', color='red', alpha=0.5)
plt.plot(result.index, result['BB_Lower'], label='Lower Band', color='green', alpha=0.5)

# Fill between bands
plt.fill_between(result.index, result['BB_Upper'], result['BB_Lower'], alpha=0.1)

# Plot signals
overbought = result[result['BB_Signal'] == -1].index
oversold = result[result['BB_Signal'] == 1].index

plt.scatter(overbought, result.loc[overbought, 'Close'], 
           marker='v', color='red', label='Sell Signal')
plt.scatter(oversold, result.loc[oversold, 'Close'],
           marker='^', color='green', label='Buy Signal')

plt.title('Bollinger Bands Analysis')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()

# Plot Bandwidth (Volatility)
plt.figure(figsize=(15, 3))
plt.plot(result.index, result['BB_Width'], label='Bandwidth', color='purple')
plt.title('Bollinger Bandwidth (Volatility)')
plt.xlabel('Date')
plt.ylabel('Bandwidth')
plt.legend()
plt.grid(True)
plt.show()

# Log analysis results
bb_oversold = len(oversold)
bb_overbought = len(overbought)
avg_volatility = result['BB_Width'].mean()
logging.info(f'Bollinger Bands Analysis: {bb_oversold} oversold, {bb_overbought} overbought signals')
logging.info(f'Average volatility (bandwidth): {avg_volatility:.4f}')

# Candlestick Analysis with Dow Theory

Dow Theory is based on six main principles:
1. The market has three movements:
   - Primary (major) trends: Long-term movement (months to years)
   - Secondary (intermediate) trends: Medium-term corrections (weeks to months)
   - Minor trends: Short-term fluctuations (days to weeks)
2. Market trends have three phases:
   - Accumulation (smart money enters)
   - Public participation (trend followers enter)
   - Distribution (smart money exits)
3. Market discounts everything
4. Market indexes must confirm each other
5. Volume must confirm the trend
6. Trends exist until definitive signals prove otherwise

In [ ]:
# Import additional libraries for candlestick plotting
import mplfinance as mpf
import yfinance as yf

def analyze_trend_with_dow_theory(data):
    """
    Analyze market trend using Dow Theory principles
    
    Parameters:
    data (pd.DataFrame): DataFrame with OHLCV data
    
    Returns:
    dict: Dictionary containing trend analysis results
    """
    logging.info('Starting Dow Theory analysis')
    
    # Calculate various moving averages for trend identification
    data['SMA_20'] = data['Close'].rolling(window=20).mean()  # Short-term trend
    data['SMA_50'] = data['Close'].rolling(window=50).mean()  # Intermediate trend
    data['SMA_200'] = data['Close'].rolling(window=200).mean()  # Primary trend
    
    # Volume moving average for confirmation
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    
    # Identify trend phases
    current_close = data['Close'].iloc[-1]
    current_volume = data['Volume'].iloc[-1]
    
    # Primary trend analysis
    primary_trend = 'SIDEWAYS'
    if current_close > data['SMA_200'].iloc[-1]:
        if data['Volume'].iloc[-1] > data['Volume_MA'].iloc[-1]:
            primary_trend = 'UP'
    elif current_close < data['SMA_200'].iloc[-1]:
        if data['Volume'].iloc[-1] > data['Volume_MA'].iloc[-1]:
            primary_trend = 'DOWN'
            
    # Secondary trend analysis (using 50-day MA)
    secondary_trend = 'SIDEWAYS'
    if current_close > data['SMA_50'].iloc[-1]:
        secondary_trend = 'UP'
    elif current_close < data['SMA_50'].iloc[-1]:
        secondary_trend = 'DOWN'
        
    # Minor trend analysis (using 20-day MA)
    minor_trend = 'SIDEWAYS'
    if current_close > data['SMA_20'].iloc[-1]:
        minor_trend = 'UP'
    elif current_close < data['SMA_20'].iloc[-1]:
        minor_trend = 'DOWN'
    
    trend_analysis = {
        'primary_trend': primary_trend,
        'secondary_trend': secondary_trend,
        'minor_trend': minor_trend,
        'volume_confirms': current_volume > data['Volume_MA'].iloc[-1]
    }
    
    logging.info(f'Dow Theory Analysis Results: {trend_analysis}')
    return trend_analysis

def plot_candlestick_with_analysis(data):
    """
    Create candlestick chart with trend analysis
    
    Parameters:
    data (pd.DataFrame): DataFrame with OHLCV data
    """
    # Prepare the plot style
    mc = mpf.make_marketcolors(up='g', down='r',
                              edge='inherit',
                              wick='inherit',
                              volume='in')
    s = mpf.make_mpf_style(marketcolors=mc)
    
    # Create the candlestick plot with volume
    ap0 = mpf.make_addplot(data['SMA_20'], color='blue', width=0.5)
    ap1 = mpf.make_addplot(data['SMA_50'], color='orange', width=0.5)
    ap2 = mpf.make_addplot(data['SMA_200'], color='red', width=0.5)
    
    mpf.plot(data, type='candle', style=s, volume=True,
             title='Forex Analysis with Dow Theory',
             addplot=[ap0, ap1, ap2],
             figratio=(16,10),
             panel_ratios=(6,2))

# Example usage with sample data (replace with your actual data)
# Download sample forex data
symbol = "EURUSD=X"  # Euro/USD forex pair
start_date = "2024-01-01"
end_date = "2024-11-05"

try:
    forex_data = yf.download(symbol, start=start_date, end=end_date)
    
    # Analyze trends using Dow Theory
    trend_analysis = analyze_trend_with_dow_theory(forex_data)
    
    # Print trend analysis results
    print("\nDow Theory Trend Analysis:")
    print(f"Primary Trend: {trend_analysis['primary_trend']}")
    print(f"Secondary Trend: {trend_analysis['secondary_trend']}")
    print(f"Minor Trend: {trend_analysis['minor_trend']}")
    print(f"Volume Confirms Trend: {trend_analysis['volume_confirms']}")
    
    # Plot candlestick chart with analysis
    plot_candlestick_with_analysis(forex_data)
    
except Exception as e:
    logging.error(f"Error in forex data analysis: {str(e)}")
    print(f"Error: {str(e)}")